# GVSA Database Analysis

This notebook provides database schema exploration and basic statistics for the GVSA soccer data.

## Contents
- Database connection and schema exploration
- Entity counts and overview
- Sample data from each table
- Basic statistics with Pandas
- Data quality checks


In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Database and ORM
from pony.orm import db_session, select, count
from models import db, Season, Division, Team, TeamSeason, Match, Club
from db_pony import GVSA_Database

# Data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully")



toml.

In [ ]:
# Connect to the database
db_path = "/projects/gvsa_scrape/gvsa_data2.db"

# Explicitly bind the database (required for Jupyter notebooks)
# This ensures the database is properly bound even if cells are run multiple times
try:
    # Unbind if already bound to a different database
    if hasattr(db, 'provider') and db.provider:
        db.disconnect()
except Exception:
    pass

# Bind to the database
db.bind(provider='sqlite', filename=db_path, create_db=False)
db.generate_mapping(create_tables=False)

# Initialize database connection (for compatibility)
gvsa_db = GVSA_Database(db_path)

print(f"Connected to database: {db_path}")
print(f"Database file exists: {Path(db_path).exists()}")

# Verify connection by checking if we can access the database
with db_session:
    season_count = count(s for s in Season)
    print(f"Seasons in database: {season_count}")


## Database Schema

The database contains the following entities:
- **Season**: Soccer seasons (Fall/Spring)
- **Division**: Divisions within seasons (e.g., "U11 Boys 5th Division")
- **Club**: Soccer clubs (e.g., "NUSC", "Rapids FC")
- **Team**: Team entities that persist across seasons
- **TeamSeason**: Team participation in specific division/season with statistics
- **Match**: Soccer matches with scores and scheduling


In [ ]:
# Display database schema information
with db_session:
    print("Database Schema:")
    print("=" * 60)
    
    # Get counts for each entity
    stats = {
        'Seasons': count(s for s in Season),
        'Divisions': count(d for d in Division),
        'Clubs': count(c for c in Club),
        'Teams': count(t for t in Team),
        'Team Seasons': count(ts for ts in TeamSeason),
        'Matches': count(m for m in Match),
    }
    
    for entity, count_val in stats.items():
        print(f"{entity:20s}: {count_val:>8,}")
    
    print("=" * 60)


## Sample Data

Let's look at sample data from each table:
y

In [ ]:
# Sample Seasons
with db_session:
    print("Sample Seasons:")
    print("-" * 60)
    seasons = select(s for s in Season).order_by(Season.scraped_at.desc()).limit(5)
    for season in seasons:
        # Display just the season_name (e.g., "Fall 2025")
        print(f"  {season.season_name} ({season.season_type})")
    print()


In [ ]:
# Sample Divisions
with db_session:
    print("Sample Divisions:")
    print("-" * 60)
    divisions = select(d for d in Division).order_by(Division.division_name).limit(10)
    for div in divisions:
        print(f"  {div.division_name} - {div.season.season_name}")
    print()


In [ ]:
# Sample Clubs
with db_session:
    print("Sample Clubs:")
    print("-" * 60)
    clubs = select(c for c in Club).order_by(Club.name).limit(20)
    for club in clubs:
        team_count = count(t for t in Team if t.club == club)
        print(f"  {club.name} ({team_count} teams)")
    print()


In [ ]:
# Sample Teams with Statistics
with db_session:
    print("Sample Team Seasons (with statistics):")
    print("-" * 60)
    team_seasons = select(ts for ts in TeamSeason).order_by(
        TeamSeason.points.desc()
    ).limit(10)
    
    for ts in team_seasons:
        print(f"  {ts.team_name}")
        print(f"    Division: {ts.division.division_name}")
        print(f"    Record: {ts.wins}W-{ts.losses}L-{ts.ties}T, {ts.points} pts")
        print(f"    Goals: {ts.goals_for}GF / {ts.goals_against}GA")
        print()


In [ ]:
# Sample Matches
with db_session:
    print("Sample Matches:")
    print("-" * 60)
    matches = select(m for m in Match).order_by(Match.date.desc()).limit(10)
    
    if count(m for m in Match) > 0:
        for match in matches:
            score_str = ""
            if match.home_score is not None and match.away_score is not None:
                score_str = f" {match.home_score}-{match.away_score}"
            print(f"  {match.home_team.team_name} vs {match.away_team.team_name}{score_str}")
            print(f"    Date: {match.date}, Status: {match.status}")
            print()
    else:
        print("  No matches found in database")


## Basic Statistics with Pandas


In [ ]:
# Load TeamSeason data into pandas DataFrame
with db_session:
    team_seasons_data = []
    for ts in select(ts for ts in TeamSeason):
        team_seasons_data.append({
            'team_name': ts.team_name,
            'division': ts.division.division_name,
            'season': ts.division.season.season_name,
            'wins': ts.wins,
            'losses': ts.losses,
            'ties': ts.ties,
            'points': ts.points,
            'goals_for': ts.goals_for,
            'goals_against': ts.goals_against,
            'goal_differential': ts.goal_differential,
        })
    
    df_teams = pd.DataFrame(team_seasons_data)

print(f"Loaded {len(df_teams)} team season records")
print(f"\nDataFrame shape: {df_teams.shape}")
print(f"\nColumns: {list(df_teams.columns)}")
df_teams.head(10)


In [ ]:
# Basic statistics
print("Team Season Statistics:")
print("=" * 60)
print(df_teams[['wins', 'losses', 'ties', 'points', 'goals_for', 'goals_against', 'goal_differential']].describe())
